# Настройка Inference 

In [10]:
from transformers import AutoTokenizer, AutoModel
import transformers
from transformers import Conversation
import torch
from typing import List

In [11]:
MODEL_NAME = 'PY007/TinyLlama-1.1B-step-50K-105b'
MODEL_TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MAX_ANSWER_LEN = 250
MIN_ANSWER_LEN = 12
RETURN_SENTENCE = 1
torch.manual_seed(21)

In [12]:
MODEL = AutoModel.from_pretrained(MODEL_NAME)

In [13]:
DEVICE

'cuda'

In [48]:
chat_bot = transformers.pipeline(
    'conversational',
    # model=MODEL,
    # device_map=DEVICE,
    
    # framework='pt',
    # eos_token_id=MODEL_TOKENIZER.eos_token_id,
    # max_length=MAX_ANSWER_LEN,
    # min_length_for_response=MIN_ANSWER_LEN,
    # num_return_sequences=RETURN_SENTENCE,
    # num_workers=-1,
    # torch_dtype='auto',
    #return_full_text = False,
   # use_fast=True
    model=MODEL_NAME,
    tokenizer=MODEL_TOKENIZER,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=MAX_ANSWER_LEN,
    min_length_for_response=MIN_ANSWER_LEN,
    num_return_sequences=RETURN_SENTENCE
)

In [42]:
def conversation_loader(conv_array: List[Conversation], pipeline, prompt:str='') -> List:
    answers = []
    for conv in conv_array:
        if not len(prompt):
            reply = pipeline(conv)
        else:
            reply = pipeline(conv, context=prompt)
        answers.append(reply.messages[-1]["content"])
    return answers
        

In [29]:
def conv_creator(questions:pd.Series) -> List[Conversation]:
    conv_array = []
    for q in questions:
        conv = Conversation(q)
        conv_array.append(conv)
    return conv_array
        

# TEST inference

In [18]:
import pandas as pd


In [25]:
df = pd.read_csv('messages/negative_mess.csv', names=['message'])

In [28]:
for a in df.message:
    print(a)

This is ridiculous! My order has been stuck on 'preparing' for over 45 minutes.;
Seriously? The delivery fee is almost as much as the food itself!;
Ugh, the app keeps crashing when I try to place my order.;
Why can't I find my favorite restaurant on this delivery app anymore?;
The estimated delivery time keeps changing! Now it says it'll be another hour?;
I specifically requested utensils and napkins, and they're nowhere to be found.;
The food selection on this app is so limited compared to other services.;
There's no way to contact the driver directly through the app? That's frustrating.;
I keep getting error messages when I try to use a promo code.;
The food pictures look amazing, but the reality is so disappointing.;
I can't believe they charge extra for dipping sauces! Ridiculous.;
This delivery service always seems to forget drinks in my orders.;
The 'live tracking' feature barely works. It's impossible to see where my food is.;
I never seem to get enough delivery drivers in my ar

In [30]:
conv_array = conv_creator(df.message)

In [31]:
conv_array

[Conversation id: cd706f90-ac57-4061-8a00-bf757a13b47c
 user: This is ridiculous! My order has been stuck on 'preparing' for over 45 minutes.;,
 Conversation id: 864d716c-dd66-49ee-be10-b6676fc56f30
 user: Seriously? The delivery fee is almost as much as the food itself!;,
 Conversation id: 9c91dc48-bfc8-4ea1-8085-a16b68fc4ade
 user: Ugh, the app keeps crashing when I try to place my order.;,
 Conversation id: ef9a0ea4-a3dc-4d3a-bbb1-3069ac892d00
 user: Why can't I find my favorite restaurant on this delivery app anymore?;,
 Conversation id: 1c667365-b007-47b8-908d-766ae0ff0943
 user: The estimated delivery time keeps changing! Now it says it'll be another hour?;,
 Conversation id: d129f385-420a-45b2-be08-093ec76df8b8
 user: I specifically requested utensils and napkins, and they're nowhere to be found.;,
 Conversation id: ea1c682f-36a6-44cc-87ae-3504c78d3a0d
 user: The food selection on this app is so limited compared to other services.;,
 Conversation id: 79dc085e-7478-4de3-b093-aeb9

In [49]:
replies = conversation_loader(conv_array, chat_bot)

d:\anaconda\Lib\site-packages\transformers\models\llama\modeling_llama.py:670: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
--- Logging error ---
Traceback (most recent call last):
  File "d:\anaconda\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "d:\anaconda\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", 